Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [67]:
import requests

In [68]:
from bs4 import BeautifulSoup

In [69]:
import pandas as pd

In [70]:
import time

In [71]:
KEYWORDS = ['github', 'блог']

URL = 'http://habr.com/ru/all/'

In [72]:
req = requests.get(URL)
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')

In [73]:
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in desired for desired in KEYWORDS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='post__title_link')
                print(title_element.text, title_element.attrs.get('href'))
               # так как пост уже нам подошел - дальше нет смысла проверять хабы
                break

GitHub: библиотека для сбора SSL-сертификатов https://habr.com/ru/company/solarsecurity/blog/525156/


In [75]:
def get_all_links(url, query, pages):
    all_refs = []
    params = {
        'search_query': query
    }
    for i in range(pages):
        params['page'] = i + 1
        res = requests.get(url, params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        news_blocks = soup.find_all('article', class_='post')
        articles_intro = list(map(lambda x: x.find('a', class_='post__title_link'), news_blocks))
        a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
        all_refs += list(map(lambda x: 'http://habr.com/ru/all/' + x, a_list))
    return all_refs

all_links = get_all_links(URL, KEYWORD, 5)
all_links

AttributeError: 'NoneType' object has no attribute 'get'

In [22]:
habr_search = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    if soup.find('div', class_='b-article__publish_date'):
        date = pd.to_datetime(soup.find('div', class_='b-article__publish_date').find('time').get('datetime'), dayfirst=True).date()
    elif soup.find('time', class_='title__cake'):
        date = pd.to_datetime(soup.find('time', class_='title__cake').get('datetime'), dayfirst=True).date()
    if soup.find('h2', class_='article_name'): 
        title = soup.find('h2', class_='article_name').text
    else: 
        title = soup.find('h1', class_='article_name').text    
    text = soup.find('div', class_='article_text_wrapper').text
    row = {'date': date, 'title': title, 'text': text}
    habr_search = pd.concat([kom_news, pd.DataFrame([row])])  
habr_search

NameError: name 'all_links' is not defined

Задание 2.¶
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>